<a href="https://colab.research.google.com/github/mahb97/Wake2vec/blob/main/Wake2Vec_750_1300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
run = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
log = json.loads((run/"metrics"/"phase1_loss_log.json").read_text())
print(run.name, "step", log[-1]["step"], "loss", round(float(log[-1]["loss"]),4))

t4_1762376560 step 550 loss 3.2604


In [ ]:
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
run = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
ck = sorted(run.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)[0]
state = json.loads((ck/"trainer_state.json").read_text())
ev = [d for d in state.get("log_history", []) if "eval_loss" in d]
print(ev[-1] if ev else "no eval yet")

{'epoch': 10.527472527472527, 'eval_loss': 7.096441268920898, 'eval_runtime': 13.6439, 'eval_samples_per_second': 3.518, 'eval_steps_per_second': 0.44, 'step': 600}


snapshot

In [ ]:
# Manual snapshot
import pathlib, shutil, time
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN   = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)

def latest_ckpt_with_weights(base):
    cands = sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)
    for ck in cands:
        if (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists():
            return ck
    return None

src = latest_ckpt_with_weights(RUN)
assert src is not None, "No valid checkpoint with weights found yet."
SNAPS = RUN/"snapshots"; SNAPS.mkdir(exist_ok=True, parents=True)
dst = SNAPS/f"snap_{int(time.time())}_{src.name}"
if not dst.exists():
    shutil.copytree(src, dst)
print("[SNAP] Saved", dst)

[SNAP] Saved /content/drive/MyDrive/wake2vec/runs/t4_1762376560/snapshots/snap_1762565503_checkpoint-300


sentry mirror

In [ ]:
# newest full checkpoint + metrics
import pathlib, shutil
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN   = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
SENTRY = DRIVE/"sentry_backups"/RUN.name; SENTRY.mkdir(parents=True, exist_ok=True)

def latest_ckpt_with_weights(base):
    cands = sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)
    for ck in cands:
        if (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists():
            return ck
    return None

ck = latest_ckpt_with_weights(RUN)
if ck is None:
    print("[SENTRY] No full checkpoint yet.")
else:
    dst = SENTRY/ck.name
    if not dst.exists():
        shutil.copytree(ck, dst)
        print(f"[SENTRY] Mirrored {ck.name} → {dst}")
    else:
        print("[SENTRY] Already mirrored:", dst)

# mirror metrics JSONs
mdst = SENTRY/"metrics"; mdst.mkdir(parents=True, exist_ok=True)
for f in (RUN/"metrics").glob("*.json"):
    shutil.copy2(f, mdst/f.name)
print("[SENTRY] Metrics mirrored →", mdst)

[SENTRY] Mirrored checkpoint-300 → /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/checkpoint-300
[SENTRY] Metrics mirrored → /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/metrics


loss and checkpoint

In [ ]:
# Heartbeat
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN   = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)

# loss
mlog = RUN/"metrics"/"phase1_loss_log.json"
if mlog.exists():
    logs = json.loads(mlog.read_text())
    print(f"[LOSS] step={logs[-1]['step']}  loss={float(logs[-1]['loss']):.4f}")
else:
    print("[LOSS] no metrics yet")

# checkpoints
def scan(base):
    rows=[]
    for ck in sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1])):
        s=int(ck.name.split("-")[-1])
        rows.append((
            s,
            (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists(),
            (ck/"trainer_state.json").exists(),
            (ck/"optimizer.pt").exists(),
        ))
    return rows

print("\n[RUNS]")
for s,w,t,o in scan(RUN):
    print(f"  {s:>4}  weights={str(w):5}  state={str(t):5}  opt={str(o):5}")

SENTRY = DRIVE/"sentry_backups"/RUN.name
if SENTRY.exists():
    print("\n[SENTRY]")
    for s,w,t,o in scan(SENTRY):
        print(f"  {s:>4}  weights={str(w):5}  state={str(t):5}  opt={str(o):5}")
else:
    print("\n[SENTRY] none")

[LOSS] step=550  loss=3.2604

[RUNS]
   100  weights=True   state=True   opt=True 
   200  weights=True   state=True   opt=True 
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=True   opt=True 
   700  weights=False  state=True   opt=True 

[SENTRY]
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=False  opt=False
   700  weights=False  state=True   opt=True 


clean drive

In [ ]:
# Light flush to help sync small files
import os, time, pathlib
RUN = max((pathlib.Path("/content/drive/MyDrive/wake2vec")/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
(open(RUN/"_touch.sync","w")).write(str(time.time()))
os.sync()
print("[SYNC] touched + sync hinted")

[SYNC] touched + sync hinted


save if trainer is in scope

In [ ]:
try:
    trainer.save_model()
    if hasattr(trainer, "_save_checkpoint"):
        trainer._save_checkpoint(model=trainer.model, trial=None)
    print("[TRAINER] save requested")
except NameError:
    print("[TRAINER] No 'trainer' object in this notebook; use the mirror cell instead.")

[TRAINER] No 'trainer' object in this notebook; use the mirror cell instead.


In [ ]:
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN_ID = "t4_1762376560"
RUN = DRIVE/"sentry_backups"/RUN_ID

# Loss heartbeat
mlog = RUN/"metrics"/"phase1_loss_log.json"
if mlog.exists():
    logs = json.loads(mlog.read_text())
    print(f"[LOSS] last step={logs[-1]['step']}  loss={float(logs[-1]['loss']):.4f}")
else:
    print("[LOSS] no metrics yet")

# Checkpoint table
def scan(base):
    rows=[]
    for ck in sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1])):
        s=int(ck.name.split("-")[-1])
        rows.append((s,
                     (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists(),
                     (ck/"trainer_state.json").exists(),
                     (ck/"optimizer.pt").exists()))
    return rows

print("\n[RUNS]")
for s,w,t,o in scan(RUN):
    print(f"  {s:>4}  weights={w!s:5}  state={t!s:5}  opt={o!s:5}")

SENTRY = DRIVE/"sentry_backups"/RUN_ID
if SENTRY.exists():
    print("\n[SENTRY]")
    for s,w,t,o in scan(SENTRY):
        print(f"  {s:>4}  weights={w!s:5}  state={t!s:5}  opt={o!s:5}")
else:
    print("\n[SENTRY] none")

[LOSS] last step=550  loss=3.2604

[RUNS]
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=False  opt=False
   700  weights=False  state=True   opt=True 

[SENTRY]
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=False  opt=False
   700  weights=False  state=True   opt=True 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("Drive mounted.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted.


In [ ]:
# VERBOSE mirror of latest full checkpoint (+ metrics) to sentry_backups
import pathlib, shutil, time, os

DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
CANDIDATES = [pathlib.Path("/content/runs"), DRIVE/"runs"]

def latest_run():
    runs = []
    for root in CANDIDATES:
        if root.exists():
            for p in root.glob("t4_*"):
                try:
                    runs.append((p.stat().st_mtime, p))
                except FileNotFoundError:
                    pass
    if not runs: return None
    runs.sort(reverse=True)
    return runs[0][1]

def latest_ckpt_with_weights(run):
    if not run: return None
    cks = sorted(run.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)
    for ck in cks:
        if (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists():
            return ck
    return None

RUN = latest_run()
print("[INFO] Active run:", RUN if RUN else "none")
CK  = latest_ckpt_with_weights(RUN)
print("[INFO] Latest full ckpt:", CK if CK else "none")

if CK is None:
    print("[SENTRY] No checkpoint with weights yet — will retry after next save.")
else:
    SENTRY = DRIVE/"sentry_backups"/RUN.name
    SENTRY.mkdir(parents=True, exist_ok=True)
    DST = SENTRY/CK.name

    src_mtime = time.ctime(CK.stat().st_mtime)
    print(f"[SENTRY] Source: {CK} (mtime {src_mtime})")
    if DST.exists():
        # check if dest is older/stale by file count or mtime
        src_files = sum(1 for _ in CK.rglob("*"))
        dst_files = sum(1 for _ in DST.rglob("*"))
        print(f"[SENTRY] Already exists: {DST} (files src={src_files} dst={dst_files})")
        if dst_files < src_files:
            print("[SENTRY] Detected partial mirror; refreshing…")
            shutil.rmtree(DST)
            shutil.copytree(CK, DST)
            print("[SENTRY] Re-mirrored:", DST)
        else:
            print("[SENTRY] Mirror up-to-date.")
    else:
        shutil.copytree(CK, DST)
        print("[SENTRY] Mirrored:", DST)

    # Mirror metrics verbosely
    msrc = RUN/"metrics"
    mdst = SENTRY/"metrics"
    mdst.mkdir(parents=True, exist_ok=True)
    copied = 0
    if msrc.exists():
        for f in msrc.glob("*.json"):
            shutil.copy2(f, mdst/f.name)
            copied += 1
    print(f"[SENTRY] Metrics mirrored → {mdst} ({copied} files)")

[INFO] Active run: /content/drive/MyDrive/wake2vec/runs/t4_1762376560
[INFO] Latest full ckpt: /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-300
[SENTRY] Source: /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-300 (mtime Wed Nov  5 22:45:46 2025)
[SENTRY] Already exists: /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/checkpoint-300 (files src=12 dst=12)
[SENTRY] Mirror up-to-date.
[SENTRY] Metrics mirrored → /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/metrics (1 files)


In [ ]:
import pathlib
BASE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN = BASE/"runs"/"t4_1762376560"  # adjust if different
SENTRY = BASE/"sentry_backups"/"t4_1762376560"

def audit(root):
    print(f"\n[{root}]")
    for ck in sorted(root.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1])):
        step = int(ck.name.split("-")[-1])
        w = (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists()
        t = (ck/"trainer_state.json").exists()
        o = (ck/"optimizer.pt").exists()
        print(f"{step:>5}  weights={w:<5}  state={t:<5}  opt={o:<5}  → {ck.name}")

if RUN.exists():   audit(RUN)
if SENTRY.exists(): audit(SENTRY)


[/content/drive/MyDrive/wake2vec/runs/t4_1762376560]
  100  weights=1      state=1      opt=1      → checkpoint-100
  200  weights=1      state=1      opt=1      → checkpoint-200
  300  weights=1      state=1      opt=1      → checkpoint-300
  400  weights=0      state=1      opt=1      → checkpoint-400
  500  weights=0      state=1      opt=1      → checkpoint-500
  600  weights=0      state=1      opt=1      → checkpoint-600
  700  weights=0      state=1      opt=1      → checkpoint-700

[/content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560]
  300  weights=1      state=1      opt=1      → checkpoint-300
  400  weights=0      state=1      opt=1      → checkpoint-400
  500  weights=0      state=1      opt=1      → checkpoint-500
  600  weights=0      state=0      opt=0      → checkpoint-600
  700  weights=0      state=1      opt=1      → checkpoint-700


weights didnt save...retrain from 300

In [8]:
from pathlib import Path
from google.colab import drive
drive.mount("/content/drive")

RUN_ID = "t4_1762376560"
RESUME_FROM = Path("/content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-750-rebuilt")
DRIVE = Path("/content/drive/MyDrive/wake2vec")
LOCAL_RUN = Path(f"/content/runs/{RUN_ID}")
SENTRY = DRIVE / "sentry_backups" / RUN_ID
EMB_SNAPS = DRIVE / "emb_snaps" / RUN_ID
DATASETS = DRIVE / "datasets"

# ensure local out exists
LOCAL_RUN.mkdir(parents=True, exist_ok=True)
SENTRY.mkdir(parents=True, exist_ok=True)
EMB_SNAPS.mkdir(parents=True, exist_ok=True)

# sanity checks
for p in [DRIVE, DRIVE/"runs", DRIVE/"sentry_backups", DRIVE/"emb_snaps", DRIVE/"datasets"]:
    assert p.exists(), f"Missing required path: {p}"

assert RESUME_FROM.exists(), f"Checkpoint not found: {RESUME_FROM}"
assert (DATASETS/"train_ds").exists(), "Missing train_ds"
assert (DATASETS/"valid_ds").exists(), "Missing valid_ds"

print("[OK] Paths in place.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[OK] Paths in place.


In [2]:
# Pin + shim
import sys, subprocess, importlib, os

def pin(pkg, ver):
    try:
        m = importlib.import_module(pkg)
        print(f"[HAVE] {pkg} {m.__version__}")
    except Exception:
        pass
    print(f"[PIN] {pkg}=={ver}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", f"{pkg}=={ver}", "-q"])

pin("transformers", "4.57.1")
pin("accelerate",   "1.2.1")
pin("datasets",     "2.21.0")

import accelerate
if not hasattr(accelerate.Accelerator, "_w2v_patched"):
    _orig = accelerate.Accelerator.unwrap_model
    def _shim(self, model, *a, **kw):
        kw.pop("keep_torch_compile", None)
        return _orig(self, model, *a, **kw)
    accelerate.Accelerator.unwrap_model = _shim
    accelerate.Accelerator._w2v_patched = True
    print("[PATCH] unwrap_model shim active")

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

[HAVE] transformers 4.57.1
[PIN] transformers==4.57.1
[HAVE] accelerate 1.11.0
[PIN] accelerate==1.2.1
[HAVE] datasets 4.0.0
[PIN] datasets==2.21.0
[PATCH] unwrap_model shim active


'expandable_segments:True'

In [3]:
# check t4
import torch
print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

CUDA: True
GPU: Tesla T4


In [4]:
# resume from 750
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

from pathlib import Path
DRIVE = Path("/content/drive/MyDrive/wake2vec")
RUN_ID = "t4_1762376560"
RESUME_FROM = DRIVE/"runs"/RUN_ID/"checkpoint-750-rebuilt"
assert RESUME_FROM.exists(), f"Missing: {RESUME_FROM}"
print("[RESUME FROM]", RESUME_FROM)

LOCAL_RUN = Path("/content/runs")/RUN_ID
LOCAL_RUN.mkdir(parents=True, exist_ok=True)

SENTRY = DRIVE/"sentry_backups"/RUN_ID
SENTRY.mkdir(parents=True, exist_ok=True)

DATASETS = DRIVE/"datasets"
assert (DATASETS/"train_ds").exists() and (DATASETS/"valid_ds").exists(), "Datasets missing."

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[RESUME FROM] /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-750-rebuilt


In [12]:
# Build a valid trainer_state.json for a true resume at step 750
from pathlib import Path
import json
from transformers.trainer_callback import TrainerState

RESUME_FROM = Path("/content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-750-rebuilt")
LAST_STEP = 750
TARGET = 1300

state = TrainerState()
state.global_step = int(LAST_STEP)
state.max_steps = int(TARGET)
state.is_hyper_param_search = False
state.log_history = []
try:
    # fyi
    state.train_batch_size = int(args.per_device_train_batch_size * args.gradient_accumulation_steps)
except Exception:
    pass

ts = RESUME_FROM / "trainer_state.json"
ts.parent.mkdir(parents=True, exist_ok=True)

# HF's own saver
if hasattr(state, "save_to_json"):
    state.save_to_json(str(ts))
else:
    payload = state.to_dict() if hasattr(state, "to_dict") else {
        "best_metric": None,
        "best_model_checkpoint": None,
        "epoch": None,
        "global_step": int(state.global_step),
        "is_hyper_param_search": bool(state.is_hyper_param_search),
        "log_history": list(state.log_history),
        "max_steps": int(state.max_steps),
        "train_batch_size": getattr(state, "train_batch_size", None),
        "total_flos": getattr(state, "total_flos", 0),
    }
    ts.write_text(json.dumps(payload, indent=2))

print("[RESUME] wrote schema-correct trainer_state.json at", ts)

[RESUME] wrote schema-correct trainer_state.json at /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-750-rebuilt/trainer_state.json


In [13]:
# Meta→real load hotfix
import torch, inspect
_orig_load_sd = torch.nn.Module.load_state_dict
def _load_state_dict_assign(self, state_dict, *args, **kwargs):
    has_meta = any(getattr(p, "is_meta", False) for p in self.parameters())
    if has_meta and "assign" in inspect.signature(_orig_load_sd).parameters:
        kwargs.setdefault("assign", True)
    return _orig_load_sd(self, state_dict, *args, **kwargs)
torch.nn.Module.load_state_dict = _load_state_dict_assign
print("[META] load_state_dict hotfix active (assign=True on meta params)")

[META] load_state_dict hotfix active (assign=True on meta params)


In [15]:
# copy HF datasets from Drive
from pathlib import Path
import shutil

DRIVE = Path("/content/drive/MyDrive/wake2vec")
DATASETS_DRIVE = DRIVE / "datasets"
DATASETS_LOCAL = Path("/content/datasets")
DATASETS_LOCAL.mkdir(parents=True, exist_ok=True)

def copy_if_missing(sub):
    src = DATASETS_DRIVE / sub
    dst = DATASETS_LOCAL / sub
    if not dst.exists():
        print(f"[COPY] {src} → {dst}")
        shutil.copytree(src, dst)
    else:
        print(f"[SKIP] already local:", dst)

copy_if_missing("train_ds")
copy_if_missing("valid_ds")

# Use local path
DATASETS = DATASETS_LOCAL
print("[DATASETS] using", DATASETS)

[COPY] /content/drive/MyDrive/wake2vec/datasets/train_ds → /content/datasets/train_ds
[COPY] /content/drive/MyDrive/wake2vec/datasets/valid_ds → /content/datasets/valid_ds
[DATASETS] using /content/datasets


In [16]:
# everything on one GPU, no meta/offload
model = AutoModelForCausalLM.from_pretrained(
    str(RESUME_FROM),
    torch_dtype=torch.float32,
    device_map=None,          # <- critical
    low_cpu_mem_usage=False   # <- build real tensors, not meta
)
model.to("cuda")
model.config.use_cache = False

# tie head↔emb
with torch.no_grad():
    model.get_output_embeddings().weight = model.get_input_embeddings().weight
try:
    model.tie_weights()
except Exception:
    pass

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-750-rebuilt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

In [14]:
# p1 resume from 750
import json, shutil, time, math, torch, numpy as np, os
from pathlib import Path
from datasets import load_from_disk
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback)

# configs
FINAL_TARGET = 1300
LAST_STEP = 750
TARGET = min(FINAL_TARGET, LAST_STEP + 550)
BASE_SAVE_STEPS = 75
SWITCH_AT = 1000
LATE_SAVE_STEPS = 100

print(f"[PLAN] last={LAST_STEP} → target={TARGET} | base_save_steps={BASE_SAVE_STEPS} | late_save={LATE_SAVE_STEPS} after {SWITCH_AT}")

# helpers
def has_weights(ck_dir: Path) -> bool:
    if not ck_dir.exists(): return False
    if (ck_dir/"model.safetensors").exists() or (ck_dir/"pytorch_model.bin").exists():
        return True
    if list(ck_dir.glob("model-*-of-*.safetensors")) or list(ck_dir.glob("pytorch_model-*-of-*.bin")):
        return True
    return False

def latest_ckpt_with_weights(root: Path):
    cks = [p for p in root.glob("checkpoint-*") if p.is_dir()]
    cks = sorted(cks, key=lambda p: int(p.name.split("-")[-1]))
    cks = [p for p in cks if has_weights(p)]
    return cks[-1] if cks else None

def ensure_resume_stub(ckpt: Path, last_step: int, target: int, per_dev_bs=1, grad_accum=16, world_size=1):
    """
    Create a minimal trainer_state.json so HF Trainer can resume from a rebuilt checkpoint.
    This DOES NOT fabricate optimizer/scheduler state; it only restores the global step for
    correct eval/save/snapshot scheduling.
    """
    ts = ckpt/"trainer_state.json"
    if ts.exists():
        print("[RESUME] trainer_state.json exists — using it")
        return
    stub = {
        "best_metric": None,
        "best_model_checkpoint": None,
        "epoch": None,
        "global_step": int(last_step),
        "is_hyper_param_search": False,
        "log_history": [],
        "max_steps": int(target),
        "num_input_batches": None,
        "train_batch_size": int(per_dev_bs * grad_accum * world_size),
        "total_flos": 0
    }
    ts.write_text(json.dumps(stub, indent=2))
    print(f"[RESUME] wrote stub trainer_state.json @ {ckpt} (global_step={last_step})")

# load tokenizer/model from 750
tok = AutoTokenizer.from_pretrained(str(RESUME_FROM), use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token or "</s>"

model = AutoModelForCausalLM.from_pretrained(str(RESUME_FROM), torch_dtype=torch.float32, device_map="auto")
model.config.use_cache = False

# tie lm_head to input embeddings
with torch.no_grad():
    if model.get_output_embeddings() is not None and model.get_input_embeddings() is not None:
        model.get_output_embeddings().weight = model.get_input_embeddings().weight
try:
    model.tie_weights()
except Exception:
    pass

# datasets
train_ds = load_from_disk(str(DATASETS/"train_ds"))
valid_all = load_from_disk(str(DATASETS/"valid_ds"))
valid_ds = valid_all.select(range(min(1000, len(valid_all))))
dc = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

# callbacks
class EvalEveryNSteps(TrainerCallback):
    def __init__(self, n=200): self.n=n
    def on_step_end(self, args, state, control, **kw):
        s = int(state.global_step or 0)
        if s and s % self.n == 0:
            control.should_log = True
            control.should_evaluate = True

class SaveCadenceSwitch(TrainerCallback):
    """Adds extra saves every 100 steps after SWITCH_AT."""
    def on_step_end(self, args, state, control, **kw):
        s = int(state.global_step or 0)
        if s >= SWITCH_AT and s % LATE_SAVE_STEPS == 0:
            control.should_save = True

class SentryMirror(TrainerCallback):
    def on_save(self, args, state, control, **kw):
        try:
            cks = [p for p in LOCAL_RUN.glob("checkpoint-*") if p.is_dir()]
            if not cks: return
            ck = max(cks, key=lambda p: int(p.name.split("-")[-1]))
            if not has_weights(ck):
                print(f"[SENTRY] skip mirror, no weights in {ck.name}")
                return
            dst = SENTRY/ck.name
            if not dst.exists():
                shutil.copytree(ck, dst)
                print(f"[SENTRY] mirrored {ck.name}")
            # metrics
            msrc = LOCAL_RUN/"metrics"
            if msrc.exists():
                (SENTRY/"metrics").mkdir(parents=True, exist_ok=True)
                for f in msrc.glob("*.json"): shutil.copy2(f, (SENTRY/"metrics"/f.name))
        except Exception as e:
            print("[SENTRY] mirror failed:", e)

class EmbeddingSnap(TrainerCallback):
    def __init__(self, every=50): self.every = every
    def on_step_end(self, args, state, control, **kw):
        s = int(state.global_step or 0)
        if s and s % self.every == 0:
            try:
                emb = model.get_input_embeddings().weight.detach().cpu()
                out = (DRIVE/"emb_snaps"/RUN_ID)/f"emb_step{int(s):04d}.pt"
                out.parent.mkdir(parents=True, exist_ok=True)
                torch.save(emb, out)
                (out.parent/"heartbeat.json").write_text(json.dumps(
                    {"run_id": RUN_ID, "step": int(s), "rows": int(emb.size(0)), "dim": int(emb.size(1)), "ts": time.time()}, indent=2))
                print(f"[SNAP] embeddings → {out.name}")
            except Exception as e:
                print("[SNAP] failed:", e)

# args
args = TrainingArguments(
    output_dir=str(LOCAL_RUN),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    max_steps=TARGET,
    learning_rate=5e-4,
    warmup_ratio=0.0,
    optim="adafactor",
    logging_steps=50,
    save_strategy="steps",
    save_steps=BASE_SAVE_STEPS,
    save_total_limit=20,
    gradient_checkpointing=True,
    fp16=False, bf16=False,
    report_to=["none"],
    max_grad_norm=1.0,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=dc,
    callbacks=[EvalEveryNSteps(200), SaveCadenceSwitch(), SentryMirror(), EmbeddingSnap(50)],
)

# resume stub then train
ensure_resume_stub(
    RESUME_FROM,
    last_step=LAST_STEP,
    target=TARGET,
    per_dev_bs=args.per_device_train_batch_size,
    grad_accum=args.gradient_accumulation_steps,
    world_size=1
)

print("[GO] training… (true HF resume from checkpoint-750-rebuilt)")
t0 = time.time()
trainer.train(resume_from_checkpoint=str(RESUME_FROM))
elapsed = time.time() - t0

# save + mirror
final_dir = LOCAL_RUN/"checkpoint-final"
trainer.save_model(str(final_dir)); tok.save_pretrained(str(final_dir))

dst = SENTRY/"checkpoint-final"
if dst.exists(): shutil.rmtree(dst)
shutil.copytree(final_dir, dst)

last_ck = latest_ckpt_with_weights(LOCAL_RUN)
print("\n[DONE]")
print(" final step target      :", TARGET)
print(" elapsed (s)            :", round(elapsed, 2))
print(" save cadence (base)    :", f"every {BASE_SAVE_STEPS} steps; plus every {LATE_SAVE_STEPS} ≥ {SWITCH_AT}")
print(" last ckpt with weights :", str(last_ck) if last_ck else "N/A")
print(" sentry mirror          :", str(dst))
print(" latest emb snapshot    :", max((DRIVE/'emb_snaps'/RUN_ID).glob('emb_step*.pt'), key=lambda p: p.name) if list((DRIVE/'emb_snaps'/RUN_ID).glob('emb_step*.pt')) else 'none yet')
print(" plots/report folder    :", str(LOCAL_RUN/'plots'))
print(" tarball (after G)      :", str(DRIVE/'archives'/f'{RUN_ID}.tar.gz'))

[PLAN] last=750 → target=1300 | base_save_steps=75 | late_save=100 after 1000


Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-750-rebuilt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The model is already on multiple devices. Skipping the move to device specified in `args`.


[RESUME] trainer_state.json exists — using it
[GO] training… (true HF resume from checkpoint-750-rebuilt)


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
	logging_steps: 50 (from args) != 500 (from trainer_state.json)
	save_steps: 75 (from args) != 500 (from trainer_state.json)
	per_device_train_batch_size: 1 (from args) != 16 (from trainer_state.json)


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# Resume P1 from best valid ckpt (300) → 750
import pathlib, shutil, json, time, os, torch
from datasets import load_from_disk
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback)

DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN_ID = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime).name
DRIVE_RUN = DRIVE/"runs"/RUN_ID
LOCAL_RUN = pathlib.Path("/content/runs")/RUN_ID
SENTRY    = DRIVE/"sentry_backups"/RUN_ID
LOCAL_RUN.mkdir(parents=True, exist_ok=True); SENTRY.mkdir(parents=True, exist_ok=True)

def list_ckpts(root):
    if not root.exists(): return []
    return sorted([p for p in root.glob("checkpoint-*") if p.is_dir()],
                  key=lambda p: int(p.name.split("-")[-1]), reverse=True)

def has_weights(ck):
    return (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists()

# pick newest valid (weights) from runs or sentry (your table shows 300 is the last valid)
cands = list_ckpts(DRIVE_RUN) + list_ckpts(SENTRY)
GOOD = next((p for p in cands if has_weights(p)), None)
assert GOOD is not None, "No valid checkpoint with weights."
last_step = int(GOOD.name.split("-")[-1])
print(f"[RESUME] {RUN_ID} from {GOOD.name} (last_step={last_step})")

# seed local output_dir from that ckpt (fast, prevents partial Drive saves)
if not (LOCAL_RUN/GOOD.name).exists():
    shutil.copytree(GOOD, LOCAL_RUN/GOOD.name)
    print("[LOCAL] seeded:", (LOCAL_RUN/GOOD.name))

# tokenizer + model
from transformers import AutoTokenizer, AutoModelForCausalLM
tok = AutoTokenizer.from_pretrained(str(GOOD), use_fast=True)
if tok.pad_token_id is None: tok.pad_token = tok.eos_token or "</s>"
model = AutoModelForCausalLM.from_pretrained(str(GOOD), torch_dtype=torch.float32, device_map="auto")
model.config.use_cache = False
with torch.no_grad():
    model.get_output_embeddings().weight = model.get_input_embeddings().weight  # tie head

# datasets
train_ds = load_from_disk(str(DRIVE/"datasets"/"train_ds"))
valid_ds = load_from_disk(str(DRIVE/"datasets"/"valid_ds")).select(range(min(1000, len(load_from_disk(str(DRIVE/'datasets'/'valid_ds'))))))
dc = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

TARGET = 1300

# Callbacks
from transformers import TrainerCallback

class EvalEveryNSteps(TrainerCallback):
    def __init__(self, n=200): self.n=n
    def on_step_end(self, args, state, control, **kw):
        if state.global_step and state.global_step % self.n == 0:
            control.should_log = True
            control.should_evaluate = True

class SentryMirror(TrainerCallback):
    def on_save(self, args, state, control, **kw):
        try:
            ck = max(LOCAL_RUN.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]))
            dst = SENTRY/ck.name
            if not dst.exists():
                shutil.copytree(ck, dst)
                print(f"[SENTRY] mirrored {ck.name}")
            # metrics
            mdst = SENTRY/"metrics"; mdst.mkdir(parents=True, exist_ok=True)
            msrc = LOCAL_RUN/"metrics"
            if msrc.exists():
                for f in msrc.glob("*.json"): shutil.copy2(f, mdst/f.name)
            os.sync()
        except Exception as e:
            print("[SENTRY] mirror failed:", e)

class EmbeddingSnap(TrainerCallback):
    def __init__(self, every=50):
        self.every = every
        (DRIVE/"emb_snaps"/RUN_ID).mkdir(parents=True, exist_ok=True)
    def on_step_end(self, args, state, control, **kw):
        if state.global_step and state.global_step % self.every == 0:
            try:
                emb = model.get_input_embeddings().weight.detach().cpu()
                path = DRIVE/"emb_snaps"/RUN_ID/f"emb_step{int(state.global_step):04d}.pt"
                torch.save(emb, path)
                (DRIVE/"emb_snaps"/RUN_ID/"heartbeat.json").write_text(json.dumps(
                    {"step": int(state.global_step), "rows": int(emb.size(0)), "dim": int(emb.size(1)), "ts": time.time()}, indent=2))
                print(f"[SNAP] embeddings → {path.name}")
            except Exception as e:
                print("[SNAP] failed:", e)

# freq at 50 until 700, then 100
save_steps = 50 if last_step < 700 else 100

from transformers import TrainingArguments
args = TrainingArguments(
    output_dir=str(LOCAL_RUN),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    max_steps=TARGET,
    learning_rate=5e-4,
    warmup_ratio=0.0,
    optim="adafactor",
    logging_steps=50,
    save_strategy="steps", save_steps=save_steps, save_total_limit=20,
    gradient_checkpointing=True,
    fp16=False, bf16=False,
    report_to=["none"],
    max_grad_norm=1.0,
)

from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=dc,
    callbacks=[EvalEveryNSteps(200), SentryMirror(), EmbeddingSnap(50)],
)

trainer.train(resume_from_checkpoint=str(LOCAL_RUN/GOOD.name))
# Finalize
trainer.save_model(str(LOCAL_RUN/"checkpoint-final")); tok.save_pretrained(str(LOCAL_RUN/"checkpoint-final"))
# Mirror final
dst = SENTRY/"checkpoint-final"
if dst.exists(): shutil.rmtree(dst)
shutil.copytree(LOCAL_RUN/"checkpoint-final", dst)
print("[SENTRY] mirrored checkpoint-final")
print("[DONE] Reached", TARGET)

[RESUME] t4_1762376560 from checkpoint-300 (last_step=300)
[LOCAL] seeded: /content/runs/t4_1762376560/checkpoint-300


`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-300 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The model is already on multiple devices. Skipping the move to device specified in `args`.
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
	save_steps: 50 (from args) != 100 (from trainer_state.json)


[SNAP] embeddings → emb_step0350.pt
[SNAP] embeddings → emb_step0400.pt
[SNAP] embeddings → emb_step0450.pt
[SNAP] embeddings → emb_step0500.pt
[SNAP] embeddings → emb_step0550.pt
[SNAP] embeddings → emb_step0600.pt
[SNAP] embeddings → emb_step0650.pt
[SNAP] embeddings → emb_step0700.pt
[SNAP] embeddings → emb_step0750.pt


In [3]:
import torch
print("CUDA:", torch.cuda.is_available(), "| device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu")

CUDA: True | device: Tesla T4


In [2]:
# Pin + shim
import sys, subprocess, importlib, os

def pin(pkg, ver):
    try:
        m = importlib.import_module(pkg)
        assert m.__version__ == ver
        print(f"[OK] {pkg} {m.__version__}")
    except Exception:
        print(f"[PIN] {pkg}=={ver}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", f"{pkg}=={ver}", "-q"])
        m = importlib.import_module(pkg)
        print(f"[OK] {pkg} {m.__version__}")

pin("transformers", "4.57.1")
pin("accelerate",   "1.2.1")
pin("datasets",     "2.21.0")

# unwrap_model shim (ignore keep_torch_compile kw)
import accelerate
if not hasattr(accelerate.Accelerator, "_w2v_patched"):
    _orig = accelerate.Accelerator.unwrap_model
    def _shim(self, model, *args, **kw):
        kw.pop("keep_torch_compile", None)
        return _orig(self, model, *args, **kw)
    accelerate.Accelerator.unwrap_model = _shim
    accelerate.Accelerator._w2v_patched = True
    print("[PATCH] unwrap_model shim active")

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

[OK] transformers 4.57.1
[PIN] accelerate==1.2.1
[OK] accelerate 1.11.0
[PIN] datasets==2.21.0
[OK] datasets 4.0.0
[PATCH] unwrap_model shim active


'expandable_segments:True'

In [ ]:
# Force resume from rebuilt 750
RUN_ID = "t4_1762376560"
RESUME_FROM = f"/content/drive/MyDrive/wake2vec/runs/{RUN_ID}/checkpoint-750-rebuilt"
print("[RESUME FROM]", RESUME_FROM)

FINAL_TARGET = 1300
last_step = 750
TARGET = min(FINAL_TARGET, last_step + 300)
save_steps = 75 if last_step < 1000 else 100
print(f"[PLAN] last={last_step} → target={TARGET} | save_steps={save_steps}")

RESUME_FROM = f"/content/drive/MyDrive/wake2vec/runs/{RUN_ID}/checkpoint-750-rebuilt"
print("[RESUME FROM]", RESUME_FROM)

for p1 eval

In [ ]:
print("TRAIN output_dir:", trainer.args.output_dir)
print("RUN_ID:", pathlib.Path(trainer.args.output_dir).name)
print("global_step:", trainer.state.global_step)

In [ ]:
import inspect
from transformers import TrainingArguments
'evaluation_strategy' in inspect.signature(TrainingArguments.__init__).parameters

In [ ]:
from transformers import TrainerCallback

class EvalEveryNSteps(TrainerCallback):
    def __init__(self, n=200): self.n=n
    def on_step_end(self, args, state, control, **kw):
        if state.global_step and (state.global_step % self.n == 0):
            control.should_save = True         # ensure state gets flushed
            control.should_log = True
            control.should_evaluate = True

In [ ]:
# P1 finalise overlap@5, norm stats, and a loss plot
import json, numpy as np, pathlib, matplotlib.pyplot as plt

DRIVE_ROOT = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUNS = sorted((DRIVE_ROOT/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
RUN_DIR = pathlib.Path("/content/runs")/RUNS[-1].name
METRICS_DIR = RUN_DIR/"metrics"
PLOTS_DIR = RUN_DIR/"plots"; PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# Load current embeddings
from transformers import AutoModelForCausalLM
BASE_CKPT = RUN_DIR/"checkpoint-final"
model = AutoModelForCausalLM.from_pretrained(str(BASE_CKPT), torch_dtype="float32", device_map=None)
E_post = model.get_input_embeddings().weight.detach().cpu().numpy()

# Optional composed init + ids
E_COMP = DRIVE_ROOT/"E_comp.npy"
NEW_IDS = DRIVE_ROOT/"new_ids.npy"
has_comp = E_COMP.exists() and NEW_IDS.exists()

def topk_overlap(a, b, k=5):
    import numpy as np
    from numpy.linalg import norm
    a = a / (norm(a, axis=1, keepdims=True)+1e-9)
    b = b / (norm(b, axis=1, keepdims=True)+1e-9)
    sims = a @ b.T
    top_a = np.argsort(-sims, axis=1)[:, :k]
    top_b = np.argsort(-sims, axis=1)[:, :k]
    inter = np.array([len(set(top_a[i]) & set(top_b[i])) for i in range(a.shape[0])])
    return inter.mean()

report = {}

if has_comp:
    E_comp = np.load(E_COMP)
    new_ids = np.load(NEW_IDS)
    E_post_new = E_post[new_ids]
    overlap5 = topk_overlap(E_comp, E_post_new, k=5)
    # Norm drift
    from numpy.linalg import norm
    dn = (norm(E_post_new, axis=1) - norm(E_comp, axis=1)).mean()
    report.update({"overlap_at_5": float(overlap5), "mean_delta_norm": float(dn), "n_new": int(len(new_ids))})
else:
    # Fallback
    from numpy.linalg import norm
    norms = norm(E_post, axis=1)
    report.update({"post_mean_norm": float(norms.mean()), "post_std_norm": float(norms.std()), "n_vocab": int(E_post.shape[0])})

# JSON
(METRICS_DIR/"p1_summary.json").write_text(json.dumps(report, indent=2))
print("[P1 SUMMARY]", json.dumps(report, indent=2))

# Loss plot
import json, glob
state_files = [RUN_DIR/"trainer_state.json", BASE_CKPT/"trainer_state.json"]
state_files = [p for p in state_files if p.exists()]
logs = []
for sf in state_files:
    s = json.loads(sf.read_text())
    logs.extend([d for d in s.get("log_history", []) if "loss" in d])

if logs:
    steps = [d["step"] for d in logs]
    losses = [float(d["loss"]) for d in logs]
    ema = []
    alpha = 0.1
    for i,x in enumerate(losses):
        ema.append(x if i==0 else alpha*x + (1-alpha)*ema[-1])
    plt.figure(figsize=(7,4.5))
    plt.plot(steps, losses, label="loss")
    plt.plot(steps, ema, label="EMA(0.1)")
    plt.title(f"P1 Loss — {RUN_DIR.name}")
    plt.xlabel("step"); plt.ylabel("loss"); plt.grid(True, linewidth=0.3); plt.legend()
    outp = PLOTS_DIR/"p1_loss_curve.png"
    plt.savefig(outp, dpi=140, bbox_inches="tight")
    print("[PLOT]", outp)
else:
    print("[WARN] No trainer_state logs found; skip loss plot.")